# The Gifting & Northern Dreaming

In [1]:
import os

import pandas as pd
import ticket_helpers
from events import (filter_by_project, filter_community_events,
                    filter_public_events, filter_school_events, load_raw_data,
                    prepare, save_files)

Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/the-gifting/_data/events'

The criteria are:

* project type is 'Produce (Signature)'
* event_type contains 'Public Event -'

It's then filtered by project_name is "07 - Children's Day"

In [3]:
data = load_raw_data().pipe(filter_by_project, "13 - Closing")
data = data.loc[data.event_type.notna()]

Run the processing and save the files

In [4]:
data.pipe(prepare).pipe(save_files, OUT_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
1289,6.0,NaN,2179,2023-10-30,Northern Dreaming Storytime Tour: Beeston Library,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS11 8PN'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Beeston and Holbeck'],Beeston and Holbeck,E05012647,6.0
1290,14.0,NaN,2199,2023-10-30,Northern Dreaming Storytime Tour: Garforth Com...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS25 1EH'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Garforth and Swillington'],Garforth and Swillington,E05013830,14.0
1291,20.0,NaN,2189,2023-10-30,Northern Dreaming Storytime Tour: Hunslet Comm...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS10 2NS'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Hunslet and Riverside'],Hunslet and Riverside,E05011399,20.0
1292,11.0,NaN,2200,2023-10-30,Northern Dreaming Storytime Tour: Rothwell Com...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,[' LS26 0AD'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Rothwell'],Rothwell,E05011410,11.0
1293,25.0,NaN,2306,2023-10-30,Northern Dreaming Storytime Tour: Steinbeck Co...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS7 2EA'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Moortown'],Moortown,E05011405,25.0
1294,20.0,NaN,2190,2023-10-30,Northern Dreaming Storytime Tour: The Den Head...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS6 2UE'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Headingley and Hyde Park'],Headingley and Hyde Park,E05011397,20.0
1295,61.0,NaN,2180,2023-10-30,Northern Dreaming Storytime Tour: The Tetley,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS10 1JQ'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Hunslet and Riverside'],Hunslet and Riverside,E05011399,61.0
1296,50.0,NaN,2170,2023-10-30,Northern Dreaming Storytime Tour: Trinity Hub,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS1 5AT'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,"ev_3134526, ev_3134527, ev_3134528",['Little London and Woodhouse'],Little London and Woodhouse,E05011403,50.0
1316,90.0,NaN,2174,2023-11-06,Northern Dreaming Storytime Tour: Austhorpe Pr...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,90.0
1317,50.0,NaN,2376,2023-11-06,Northern Dreaming Storytime Tour: Beechwood Pr...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,50.0


Filter by public events

In [5]:
PUBLIC_EVENTS_DIR = OUT_DIR+'/public'
os.makedirs(PUBLIC_EVENTS_DIR, exist_ok=True)


data.pipe(filter_public_events).pipe(prepare).pipe(save_files, PUBLIC_EVENTS_DIR)
data

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue)
1289,6.0,NaN,2179,2023-10-30,Northern Dreaming Storytime Tour: Beeston Library,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS11 8PN'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Beeston and Holbeck']
1290,14.0,NaN,2199,2023-10-30,Northern Dreaming Storytime Tour: Garforth Com...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS25 1EH'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Garforth and Swillington']
1291,20.0,NaN,2189,2023-10-30,Northern Dreaming Storytime Tour: Hunslet Comm...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS10 2NS'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Hunslet and Riverside']
1292,11.0,NaN,2200,2023-10-30,Northern Dreaming Storytime Tour: Rothwell Com...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,[' LS26 0AD'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Rothwell']
1293,25.0,NaN,2306,2023-10-30,Northern Dreaming Storytime Tour: Steinbeck Co...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS7 2EA'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Moortown']
1294,20.0,NaN,2190,2023-10-30,Northern Dreaming Storytime Tour: The Den Head...,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS6 2UE'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Headingley and Hyde Park']
1295,61.0,NaN,2180,2023-10-30,Northern Dreaming Storytime Tour: The Tetley,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS10 1JQ'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,NaN,['Hunslet and Riverside']
1296,50.0,NaN,2170,2023-10-30,Northern Dreaming Storytime Tour: Trinity Hub,['Public Event - Performance'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,['LS1 5AT'],13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-10-30,"ev_3134526, ev_3134527, ev_3134528",['Little London and Woodhouse']
1316,90.0,NaN,2174,2023-11-06,Northern Dreaming Storytime Tour: Austhorpe Pr...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN
1317,50.0,NaN,2376,2023-11-06,Northern Dreaming Storytime Tour: Beechwood Pr...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN


Filter by Community events

In [6]:
# COMMUNITY_EVENTS_DIR = OUT_DIR+'/community'
# os.makedirs(COMMUNITY_EVENTS_DIR, exist_ok=True)

# data.pipe(filter_community_events).pipe(prepare).pipe(save_files, COMMUNITY_EVENTS_DIR)

Filter by Schools events

In [7]:
SCHOOLS_EVENTS_DIR = OUT_DIR+'/schools'
os.makedirs(SCHOOLS_EVENTS_DIR, exist_ok=True)


data.pipe(filter_school_events).pipe(prepare).pipe(save_files, SCHOOLS_EVENTS_DIR)

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue),ward_name,ward_code,audience_size
1316,90.0,NaN,2174,2023-11-06,Northern Dreaming Storytime Tour: Austhorpe Pr...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,90.0
1317,50.0,NaN,2376,2023-11-06,Northern Dreaming Storytime Tour: Beechwood Pr...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,50.0
1318,90.0,NaN,2178,2023-11-06,Northern Dreaming Storytime Tour: Birchfield P...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,90.0
1319,140.0,NaN,2186,2023-11-06,Northern Dreaming Storytime Tour: Blackgates P...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,140.0
1320,180.0,NaN,2198,2023-11-06,Northern Dreaming Storytime Tour: Churwell Pri...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,180.0
1321,180.0,NaN,2175,2023-11-06,Northern Dreaming Storytime Tour: Clapgate Pri...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,180.0
1322,135.0,NaN,2193,2023-11-06,Northern Dreaming Storytime Tour: Drighlington...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,135.0
1323,90.0,NaN,2194,2023-11-06,Northern Dreaming Storytime Tour: Elements Pri...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,90.0
1324,100.0,NaN,2177,2023-11-06,Northern Dreaming Storytime Tour: Fountain Pri...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,100.0
1325,90.0,NaN,2197,2023-11-06,Northern Dreaming Storytime Tour: Gildersome P...,['CLE - Creative Learning session'],13 - Closing: Northern Dreaming Storytime Tour...,NaN,NaN,13 - Closing,Produce (Signature),['Season 3 - Dreaming'],2023-11-06,NaN,NaN,<NA>,NaN,90.0


## Ticket Processing

In [8]:
ticket_sales = (
    ticket_helpers
        .get_tickets_for_project('13 - Closing')
        .pipe(ticket_helpers.filter_public_events)
)

In [9]:
# ticket_sales = data.pipe(filter_public_events).pipe(ticket_helpers.get_tickets_for_event)

By ward

In [10]:
ticket_sales.pipe(ticket_helpers.summarise_by_ward).reset_index().to_csv(
    os.path.join(OUT_DIR, 'tickets_by_ward.csv'), index=False)

By date

In [11]:
(
  ticket_sales
    .pipe(ticket_helpers.summarise_by_date)
    .resample('W-FRI').sum()
    .cumsum().to_csv(os.path.join(OUT_DIR, 'tickets_by_date.csv'))
)

In [12]:
def add_human_date(data):
    datetime = data.index.to_series().dt.strftime('%a %d %b, %l:%M %P').rename('human_datetime')
    return pd.concat([data, datetime], axis=1)

ticket_sales.loc[
  (ticket_sales.event_datetime > pd.to_datetime('2023-12-21')) & (ticket_sales.public_event == True)
].groupby('event_datetime').number_of_tickets.sum().sort_index().pipe(add_human_date).to_csv(f'{OUT_DIR}/tickets_by_event_datetime.csv')